<a href="https://colab.research.google.com/github/Drozeler/Machine-Learning/blob/main/NLPChatBotSentimentAnalysisWeather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy transformers
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from transformers import pipeline
import requests
import datetime
import logging

In [ ]:
nlp = spacy.load("en_core_web_sm")
sentiment_analyzer = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that aut

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
logging.basicConfig(filename="chatbot.log", level=logging.INFO, format="%(asctime)s - %(message)s")
def get_greeting():
  """Generate a greeting based on the time of day."""
  current_hour = datetime.datetime.now().hour
  if current_hour < 12:
    return "Good morning!"
  elif 12 <= current_hour <18:
    return "Good afternoon!"
  else:
    return "Good evening!"

def get_weather(location):
  """Get weather information for a given location using an API."""
  api_key = "e6540cd3f9901379f03f969c82eb6cbf"
  url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}"
  try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    weather = data["weather"][0]["description"]
    temp = data["main"]["temp"]
    return f"The current weather in {location} is {weather} with a temperature of {temp-273}°C."
  except requests.exceptions.RequestException as e:
    return f"Sorry, I couldn't fetch the weather information right now. Error: {e}"

def analyze_sentiment(text):
  """Analyze the sentiment of the given text."""
  result = sentiment_analyzer(text)[0]
  return result['label'], result['score']

def respond_to_input(user_input, context):
  """Generate a response based on user input and context."""
  doc = nlp(user_input)
  response = "I'm not sure how to respond to that."

  if "name" in [token.text.lower() for token in doc]:
    response = "I'm your friendly chatbot created to assist you."
  elif 'weather' in [token.text.lower() for token in doc]:
    location = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
    if location:
      response = get_weather(location[0])
    else:
      response = "Please provide a location for the weather update"
  elif "how are you" in user_input.lower():
    response = "I'm just a program, but I'm here to help you!"
  else:
    sentiment, score = analyze_sentiment(user_input)
    if sentiment == "POSITIVE":
      response = "I'm glad to hear that!"
    elif sentiment == "NEGATIVE":
      response = "I'm sorry to hear that. How can I assist you?"
    else:
      response = "Thank you for sharing that."

  context.append((user_input, response))
  return response

def main():
  """Main function to run the chatbot."""
  print(get_greeting(), "I'm your enhanced chatbot! Type 'quit' to exit.")
  context = []
  while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["quit", "bye", "exit"]:
      print("Chatbot: Goodbye! Have a great day!")
      break
    response = respond_to_input(user_input, context)
    print("Chatbot:", response)
    logging.info(f"User: {user_input} | Chatbot: {response}")


if __name__ == "__main__":
  main()


Good morning! I'm your enhanced chatbot! Type 'quit' to exit.
You: I love this chatbot
Chatbot: I'm glad to hear that!
You: Jakarta weather
Chatbot: The current weather in Jakarta is haze with a temperature of 30.319999999999993°C.
You: Man its so hot
Chatbot: I'm sorry to hear that. How can I assist you?
You: i dont know
Chatbot: I'm sorry to hear that. How can I assist you?
You: .
Chatbot: I'm glad to hear that!
You: lmao
Chatbot: I'm glad to hear that!
You: quit
Chatbot: Goodbye! Have a great day!
